In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
FRmodel = faceRecoModel2(input_shape=(3, 96, 96), input_shape2 = (3, 96, 96))

In [3]:


def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    pos_dist = tf.square(tf.subtract(anchor, positive))
    neg_dist = tf.square(tf.subtract(anchor, negative))
    basic_loss = tf.add(tf.reduce_sum(tf.subtract(pos_dist,neg_dist)), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.))
    
    return loss

In [4]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean = 6, stddev = 0.1, seed =1),
              tf.random_normal([3, 128], mean = 1, stddev = 1, seed = 1),
              tf.random_normal([3, 128], mean = 3, stddev = 4, seed = 1))
    loss = triplet_loss(y_true, y_pred, 0.2)
    
    print("loss = " + str(loss.eval()))

loss = 350.02637


In [18]:
FRmodel.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
load_weights_from_FaceNet(FRmodel)

In [6]:
database = {}
database["ivan"] = img_to_encoding("images/myImages/ivan.jpg", FRmodel)
database["ivan2"] = img_to_encoding("images/myImages/ivan2.jpg", FRmodel)
database["alinka"] = img_to_encoding("images/myImages/alinka.jpg", FRmodel)
database["danil"] = img_to_encoding("images/myImages/danil.jpg", FRmodel)
database["pasha"] = img_to_encoding("images/myImages/pasha.jpg", FRmodel)
database["sasha"] = img_to_encoding("images/myImages/sasha.jpg", FRmodel)
database["zjenya"] = img_to_encoding("images/myImages/zjenya.jpg", FRmodel)
database["slava"] = img_to_encoding("images/myImages/slava.jpg", FRmodel)

In [29]:
names = ["ivan", "ivan2", "alinka", "danil", "pasha", "sasha", "zjenya", "slava"]

In [25]:
result = img_to_encoding("images/myImages/ivan.jpg", "images/myImages/pasha.jpg", FRmodel)

In [26]:
print(result)

[[0.53262645]]


In [11]:
#x_train2 = [take_images("images/myImages/" + name + ".jpg") for name in names]
x_train2 = [take_images("images/myImages/ivan.jpg")]
x_train2_database = [take_images("images/myImages/ivan2.jpg")]

In [49]:
x = x_train2[0][1]
print("x_train2: {0}\ntype1: {1}\ntype2: {2}\nshape: {3}\n".format(x, type(x_train2), type(x), "None" if type(x) != np.ndarray else x.shape))


x_train2: [[[[0.47058824 0.54117647 0.79215686 0.88235294 0.89411765 1.
    0.98823529 0.99607843 0.94901961 0.6627451  0.76862745 0.90588235
    0.90588235 0.84705882 0.56862745 0.61176471 0.6        0.64705882
    0.98431373 0.72156863 0.38823529 0.32156863 0.3254902  0.25882353
    0.34117647 0.45490196 0.2627451  0.25882353 0.25882353 0.20784314
    0.17254902 0.25882353 0.21568627 0.23529412 0.25882353 0.22352941
    0.24313725 0.17647059 0.17647059 0.19215686 0.17647059 0.21568627
    0.26666667 0.21960784 0.17254902 0.19215686 0.25882353 0.22352941
    0.30588235 0.3372549  0.30196078 0.31764706 0.28235294 0.40784314
    0.59215686 0.63529412 0.64313725 0.69803922 0.66666667 0.72156863
    0.7254902  0.7372549  0.76470588 0.78039216 0.78431373 0.74117647
    0.7372549  0.76862745 0.74509804 0.73333333 0.72941176 0.72156863
    0.72941176 0.66666667 0.67058824 0.70196078 0.7254902  0.75294118
    0.76470588 0.78039216 0.78039216 0.78039216 0.76862745 0.78431373
    0.78039216 0.7

In [33]:
x_train = [img for img in database.values()]
y_train = [key for key in database.keys()]
x = x_train[1]
y = y_train[1]
print("x_train: {0}\ntype1: {1}\ntype2: {2}\nshape: {3}\n".format(x, type(x_train), type(x), "None" if type(x) != np.ndarray else x.shape))
print("y_train: {0}\ntype1: {1}\ntype2: {2}\nshape: {3}\n".format(y, type(y_train), type(y), "None" if type(y) != np.ndarray else y.shape))


x_train: [[ 4.18480039e-02 -2.34748283e-03  1.51765332e-01  1.72788560e-01
   8.11235681e-02  1.74577072e-01  3.42675187e-02 -1.65913895e-01
   9.80474800e-02 -1.32491991e-01 -6.81956112e-02  1.37238465e-02
  -8.02081525e-02  2.81369705e-02  1.19958416e-01 -7.47915208e-02
  -6.40267599e-03 -3.49098211e-03 -4.85022925e-02 -6.04589768e-02
  -3.11780721e-02  3.38568054e-02  5.24521247e-02  1.47793740e-01
   1.28512472e-01 -1.73504651e-01 -1.26160085e-01 -3.15536261e-02
  -1.09191164e-01 -2.56673712e-02  1.67403609e-01  5.10891005e-02
  -1.01604626e-01  4.12445962e-02  7.51796663e-02  4.82741147e-02
   4.40482609e-02 -1.84707139e-02 -3.69359888e-02 -3.66568789e-02
   1.52872503e-01  4.52847369e-02 -2.89918985e-02 -2.19277576e-01
  -5.96299954e-02 -5.27629331e-02 -5.85353598e-02 -3.87861401e-05
   7.71818981e-02  6.75808452e-03  4.15518396e-02 -7.29722157e-02
   1.99633434e-01 -3.59246507e-02  4.81018908e-02  2.28455029e-02
  -2.22004112e-02  9.62995887e-02  8.82511958e-03 -3.80029008e-02
 

In [12]:
y_train2 = [(1)]

In [15]:
FRmodel.fit([x_train2, x_train2_database], y_train2)

ValueError: Error when checking input: expected input_7 to have 4 dimensions, but got array with shape (1, 1, 3, 96, 96)

In [52]:


def verify(image_path, identity, database, model):
    encoding = img_to_encoding(image_path, model)
    dist = np.linalg.norm(encoding-database[identity])
    
    if dist < 0.7:
        print("Hi, Vanchez")
        door_open = True
    else:
        print("Go away!! You are not Vanchez!")
        door_open = False
    
    return dist, door_open

In [63]:
verify("images/myImages/camera_ivan2.jpg", "ivan2", database, FRmodel)

Hi, Vanchez


(0.0, True)

In [54]:


def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding-db_enc)
        
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print("It's " + str(identity) + ", the distance is " + str(min_dist))
    
    return min_dist, identity

In [64]:
who_is_it("images/myImages/camera_ivan2.jpg", database, FRmodel)

It's ivan2, the distance is 0.0


(0.0, 'ivan2')